# Ensemble Learning

## Initial Imports

In [157]:
import warnings
warnings.filterwarnings('ignore')

In [158]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [159]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [160]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [161]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
label_encoder = LabelEncoder()
label_encoder.fit(df["loan_status"])
df["loan_status"] = label_encoder.transform(df["loan_status"])

In [162]:
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,1,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,1,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,1,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,1,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,1,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [163]:
# Binary encoding using Pandas (single column)
loans_binary_encoded = pd.get_dummies(df, columns=["home_ownership","verification_status","issue_d","next_pymnt_d","pymnt_plan","initial_list_status","application_type","hardship_flag","debt_settlement_flag"])
loans_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,1,0,1,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,1,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,1,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,1,31.44,0.0,1.0,10.0,1.0,...,1,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,1,18.76,0.0,1.0,14.0,0.0,...,1,0,1,1,0,1,1,0,1,1


In [164]:
loans_binary_encoded.dtypes

loan_amnt                              float64
int_rate                               float64
installment                            float64
annual_inc                             float64
loan_status                              int64
dti                                    float64
delinq_2yrs                            float64
inq_last_6mths                         float64
open_acc                               float64
pub_rec                                float64
revol_bal                              float64
total_acc                              float64
out_prncp                              float64
out_prncp_inv                          float64
total_pymnt                            float64
total_pymnt_inv                        float64
total_rec_prncp                        float64
total_rec_int                          float64
total_rec_late_fee                     float64
recoveries                             float64
collection_recovery_fee                float64
last_pymnt_am

## Split the Data into Training and Testing

In [165]:
# Create our features
# YOUR CODE HERE
X= loans_binary_encoded.drop(columns="loan_status")
# Create our target
# YOUR CODE HERE
y=loans_binary_encoded["loan_status"]


In [166]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,0.383161,0.616839,1.0,0.123879,0.876121,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.486161,0.486161,0.0,0.329446,0.329446,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,1.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,1.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [167]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [168]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [169]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
datascaler = StandardScaler()

In [170]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = datascaler.fit(X_train)

In [171]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [172]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [173]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_rf = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_rf)
#print(bas_brf)

0.7940331008698245

In [174]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_rf)


array([[   60,    27],
       [ 1739, 15379]])

In [175]:
# Print the imbalanced classification report
# YOUR CODE HERE
from sklearn.metrics import classification_report
#y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.69      0.90      0.06      0.79      0.61        87
          1       1.00      0.90      0.69      0.95      0.79      0.63     17118

avg / total       0.99      0.90      0.69      0.94      0.79      0.63     17205



In [176]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE

importances = brf.feature_importances_

sorted_indices = np.argsort(importances)[::-1]

print(*X_train.columns[sorted_indices], sep = "\n")


total_rec_prncp
total_pymnt
total_rec_int
last_pymnt_amnt
total_pymnt_inv
int_rate
issue_d_Jan-2019
out_prncp
dti
out_prncp_inv
revol_bal
mo_sin_old_rev_tl_op
mths_since_recent_inq
annual_inc
il_util
bc_util
issue_d_Mar-2019
all_util
installment
max_bal_bc
mths_since_recent_bc
mo_sin_old_il_acct
total_bal_il
avg_cur_bal
bc_open_to_buy
tot_cur_bal
total_bal_ex_mort
tot_hi_cred_lim
total_bc_limit
num_rev_accts
mths_since_rcnt_il
total_il_high_credit_limit
total_rev_hi_lim
mo_sin_rcnt_rev_tl_op
num_op_rev_tl
loan_amnt
num_sats
inq_last_12m
pct_tl_nvr_dlq
num_bc_tl
num_bc_sats
total_cu_tl
num_actv_rev_tl
total_acc
percent_bc_gt_75
num_il_tl
mo_sin_rcnt_tl
acc_open_past_24mths
open_acc
num_tl_op_past_12m
num_rev_tl_bal_gt_0
num_actv_bc_tl
open_acc_6m
inq_fi
open_rv_24m
open_act_il
mort_acc
next_pymnt_d_May-2019
inq_last_6mths
open_il_24m
total_rec_late_fee
open_il_12m
open_rv_12m
issue_d_Feb-2019
delinq_2yrs
next_pymnt_d_Apr-2019
verification_status_Verified
num_accts_ever_120_pd
tot_coll_a

### Easy Ensemble Classifier

In [177]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [178]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_eec = eec.predict(X_test_scaled)
bas_eec=balanced_accuracy_score(y_test, y_pred_eec)
print(bas_eec)

0.9251352679776481


In [179]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_eec)


array([[   79,     8],
       [  989, 16129]])

In [180]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.07      0.91      0.94      0.14      0.92      0.85        87
          1       1.00      0.94      0.91      0.97      0.92      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.92      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classification has the best accuracy score at 0.9251352679776481 while the Balanced Random Forest accuracy score is only 0.7940331008698245

2. Which model had the best recall score?

    The Easy Ensemble Classification has the best recall score at 0.94 while Balanced Random Forest recall is only at 0.90.

3. Which model had the best geometric mean score?

    The Easy Ensemble Classification has the best recall score at 0.92 while Balanced Random Forest recall is only at 0.79.

4. What are the top three features?

    total_rec_prncp
    total_pymnt
    total_rec_int